In [1]:
import pandas as pd
import os
import pickle
import time
from pathlib import Path
import datetime as dt
from datetime import datetime
from datetime import timedelta
import pandas.io.parsers

In [2]:
instrument = ("BANKNIFTY").upper()

start_date = datetime.strptime("2023-01-01", "%Y-%m-%d").date()
end_date = datetime.strptime("2023-02-28", "%Y-%m-%d").date()

data_root_path = Path(r"D:\Data\Dataset\Dict")
output_csv_path = Path(r"C:\Users\ABC\OneDrive\Desktop")
param_file_path = Path(r"C:\Users\ABC\OneDrive\Desktop\Raghav_Param.csv")

param_data = pd.read_csv(param_file_path)
# param_data = param_data[(param_data.Entry_time == '11:00:59') & (param_data.Multiplier == 0.5) &
#                        (param_data.Roundoff == 50) & (param_data.Stoploss == 20)].head(1)
param_data = param_data.head(1)
param_data

,Entry_Time,Exit_Time,Entry_Weekday,StopLoss,Max_Entries
0,09:20:59,15:20:59,Friday,20,5


In [3]:
# Pre Defined Functions

# Roundoff : This Funtion is Using A Value Which your Want to Convert in Whole Number of its given input Value (50, 100)
def Roundoff(x, base=0.05):
    return base * round(x / base)

# Filename :  This Function Creates a Filename based on your Date
""" In Our System -  For Pickle Dataframe : File Name format is (BN_01012021.pkl / NIFTY_01012021.pkl), it contains Day wise Data
                    For Pickle Dictionary : File Name Format is (BN_2021.pkl / NIFTY_2021.pkl), it contains yearly Data"""
def Filename(year, Instrument):
    year = year
    if Instrument == "BANKNIFTY":
        filename = f"BN_{year}.pkl"
    else:
        filename = f"{Instrument}_{year}.pkl"
    return filename, year

In [4]:
dates_range = [str(start_date+timedelta(days=x)) for x in range((end_date-start_date).days + 1)]
year_data = str(start_date.year)

In [5]:
column_list = ['User Entry Time', 'User Exit Time', 'user_weekday', 'Stop Loss %', 'Max Entries', 'Expiry','Entry Date_1', 'Exit Date_1', 'Entry Weekday_1', 'Exit Weekday_1', 'Entry Time_1' ,'Exit Time_1', 'Exit Straddle_1', 'Stop Loss_1', 'SL Type_1' ,'CE_PNL_1', 'PE_PNL_1', 'Total_PNL_1']
market_opening_time = '09:15:59'
market_closing_time = '15:29:59'

In [6]:
def Entry_Function(df, expiry, entry_time, user_weekday, stop_loss_p):
    weekdays_name_list = ['Friday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday']
    dates_keys_list = sorted(list(df[expiry].keys()))
    weekday_list = list(map(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%A'), dates_keys_list))
    try:
        start_index = weekday_list.index(user_weekday)
    except:
        try:
            weekday_index = weekdays_name_list.index(user_weekday)
            weekday_next_day = weekdays_name_list[weekday_index + 1]
            start_index = weekday_list.index(weekday_next_day)
        except:
            weekday_index = weekdays_name_list.index(user_weekday)
            weekday_next_day = weekdays_name_list[weekday_index + 2]
            start_index = weekday_list.index(weekday_next_day)
    date_key_list = dates_keys_list[start_index : ]
    entry_date = date_key_list[0]
    entry_weekday = (pd.Timestamp(entry_date)).day_name()
    
    if entry_time in (df[expiry][entry_date].keys()):
        ce_strk_set = list((df[expiry][entry_date][entry_time]["CE"].keys()))
        pe_strk_set = list((df[expiry][entry_date][entry_time]["PE"].keys()))
        common_strike = list(set(ce_strk_set) & set(pe_strk_set))
        price_diff, strikes_list = [], []
        for strk in common_strike:
            price_diff.append(abs((df[expiry][entry_date][entry_time]["CE"][strk]['close'])-(df[expiry][entry_date][entry_time]["PE"][strk]['close'])))
            strikes_list.append(strk)
            Common_Strike_Price = strk
            ATM_strike = strikes_list[price_diff.index(min(price_diff))]
            ep_ce = df[expiry][entry_date][entry_time]["CE"][ATM_strike]['close']
            ep_pe = df[expiry][entry_date][entry_time]["PE"][ATM_strike]['close']
            ep_straddle = ep_ce + ep_pe
            SL = ep_straddle + (ep_straddle * stop_loss_p)/100
    else:
        ATM_strike, ep_ce, ep_pe, ep_straddle, SL = "", "", "", "", "" 
    return ATM_strike, ep_ce, ep_pe, ep_straddle, SL, date_key_list, entry_date, entry_weekday

In [7]:
def Rolling_Straddle_Positional(df, expiry_list, entry_time, exit_time, stop_loss_p, user_weekday, max_entries):
    mergefile = []
    user_entry_time, user_exit_time = entry_time, exit_time
    for expiry in expiry_list:
        try:
            sl_type = ""
            same_entry_list = []
            for n in range(max_entries):
                k = n + 1

                if (sl_type == "Hit") or (sl_type == ""):
                    if (sl_type == "Hit"):
                        entry_time = exit_time
                        entry_weekday = (pd.Timestamp(exit_date)).day_name()
                        ATM_strike, ep_ce, ep_pe, ep_straddle, SL, date_key_list, entry_date, entry_weekday = Entry_Function(df, expiry, entry_time, entry_weekday, stop_loss_p)
                    else:
                        entry_time = user_entry_time
                        ATM_strike, ep_ce, ep_pe, ep_straddle, SL, date_key_list, entry_date, entry_weekday = Entry_Function(df, expiry, entry_time, user_weekday, stop_loss_p)
                else:
                    entry_time = user_entry_time
                    entry_date = first_entry_date
                    ATM_strike, ep_ce, ep_pe, ep_straddle, SL, date_key_list, entry_date, entry_weekday = Entry_Function(df, expiry, entry_time, user_weekday, stop_loss_p)

                for d in date_key_list:
                    if (d == entry_date) & ((pd.Timestamp(entry_date)).day_name() == "Thursday"):
                        time_list = sorted([key for key, value in df[expiry][d].items() if entry_time < key <= exit_time])
                    elif d == entry_date:
                        time_list = sorted([key for key, value in df[expiry][d].items() if entry_time < key <= market_closing_time])             
                    elif d == date_key_list[-1]:
                        time_list = sorted([key for key, value in df[expiry][d].items() if market_opening_time < key <= exit_time])
                    else:
                        time_list = sorted([key for key, value in df[expiry][d].items() if market_opening_time < key <= market_closing_time])
                    for time in time_list:
                        try:
                            straddle = df[expiry][d][time]["CE"][ATM_strike]['close'] + df[expiry][d][time]["PE"][ATM_strike]['close']
                            if(SL <= straddle):  
                                sl_type = "Hit"
                                exit_time = time
                                exit_straddle = straddle
                                CE_PNL = ep_ce - df[expiry][d][time]["CE"][ATM_strike]['close']
                                PE_PNL = ep_pe - df[expiry][d][time]["PE"][ATM_strike]['close']
                                exit_date = d
                                exit_weekday = (pd.Timestamp(exit_date)).day_name()
                                break
                            else:
                                sl_type = "Not Hit"
                                blank = 0
                                exit_straddle = straddle
                                exit_time = time
                                CE_PNL = ep_ce - df[expiry][d][time]["CE"][ATM_strike]['close']
                                PE_PNL = ep_pe - df[expiry][d][time]["PE"][ATM_strike]['close'] 
                                exit_date = d
                                exit_weekday = (pd.Timestamp(exit_date)).day_name()
                        except:
                            sl_type, exit_time, CE_PNL, PE_PNL, exit_date, exit_weekday = "", "", "", "", "", ""
                            pass

                    if sl_type == "Hit":
                        output_df = pd.DataFrame(columns= [f'Entry Date_{k}', f'Exit Date_{k}', f'Entry Weekday_{k}', f'Exit Weekday_{k}', f'Entry Time_{k}', f'Exit Time_{k}', f'Exit Straddle_{k}', f'Stop Loss_{k}', f'SL Type_{k}', f'CE_PNL_{k}', f'PE_PNL_{k}', f'Total_PNL_{k}'])
                        c = 0
                        output_df.loc[c] = [entry_date, exit_date, entry_weekday, exit_weekday, entry_time, exit_time, exit_straddle , SL, sl_type, CE_PNL, PE_PNL, CE_PNL+PE_PNL]  
                        same_entry_list.append(output_df)
                        break
                    else:
                        pass


                if sl_type == "Not Hit":
                    if len(same_entry_list) > 0:
                        df_merged = pd.concat(same_entry_list, axis=1)
                        df_merged['User Entry Time'] =  user_entry_time 
                        df_merged['User Exit Time'] = user_exit_time 
                        df_merged['user_weekday'] = user_weekday 
                        df_merged['Stop Loss %'] = stop_loss_p
                        df_merged['Max Entries'] = max_entries
                        df_merged['Expiry'] = expiry
                        x_y_columns_list = sorted(list(set(column_list) ^ set(df_merged.columns)))
                        new_order = column_list + x_y_columns_list
                        df_merged = df_merged.reindex(columns=new_order)
                        output_df = pd.DataFrame(columns= [f'Entry Date_{k}', f'Exit Date_{k}', f'Entry Weekday_{k}', f'Exit Weekday_{k}', f'Entry Time_{k}', f'Exit Time_{k}', f'Exit Straddle_{k}', f'Stop Loss_{k}', f'SL Type_{k}', f'CE_PNL_{k}', f'PE_PNL_{k}', f'Total_PNL_{k}'])
                        c = 0
                        output_df.loc[c] = [entry_date, exit_date, entry_weekday, exit_weekday, entry_time, exit_time, exit_straddle , SL, sl_type, CE_PNL, PE_PNL, CE_PNL+PE_PNL]  
                        data = pd.concat([df_merged, output_df], axis=1)
                        mergefile.append(data)
                    else:
                        output_df = pd.DataFrame(columns= ['User Entry Time', 'User Exit Time', 'user_weekday','Stop Loss %', 'Max Entries', 'Expiry','Entry Date_1', 'Exit Date_1', 'Entry Weekday_1', 'Exit Weekday_1', 'Entry Time_1' ,'Exit Time_1', 'Exit Straddle_1', 'Stop Loss_1', 'SL Type_1' ,'CE_PNL_1', 'PE_PNL_1', 'Total_PNL_1'])
                        c = 0
                        output_df.loc[c] = [entry_time, exit_time, user_weekday, stop_loss_p, max_entries, expiry, entry_date, exit_date, entry_weekday, exit_weekday, entry_time, exit_time, exit_straddle , SL, sl_type, CE_PNL, PE_PNL, CE_PNL+PE_PNL]  
                        mergefile.append(output_df)
                    break
                else:
                    pass
        except:
            pass
    df = pd.concat(mergefile, ignore_index=True)
    df.reset_index(drop=True, inplace=True)
    display(df)  
    starting_time = (datetime.strptime(entry_time, "%H:%M:%S").time()).strftime("%H-%M-%S")
    exiting_time = (datetime.strptime(exit_time, "%H:%M:%S").time()).strftime("%H-%M-%S")

    folder_name = "Rolling_Straddle_Positional_Outputs"  
    file_starting_name = "PRolling_Straddle_Positional"  

    file_name = (f"{file_starting_name}_{starting_time}_{exiting_time}-{stop_loss_p}_{max_entries}.csv")
    output_file_path = (output_csv_path / folder_name / file_name)

    if not os.path.exists(output_csv_path / f"{folder_name}"):
        os.makedirs(output_csv_path / f"{folder_name}")

    if not os.path.exists(output_file_path):
        df.to_csv(output_file_path, mode='a', index=False, header=True)
    else: 
        df.to_csv(output_file_path, mode='a', index=False, header=False)
        read = pd.read_csv(output_file_path)
        df.to_csv(output_file_path, index=False, header=True)
    print("Donez !!!!!")   

In [8]:
def Driver_Code(option_data, expiry_key_list, param_data):
    for params in param_data.itertuples():  # iterating Csv Parameters for Strategy 
        entry_time = params.Entry_Time      # Csv Parameter
        exit_time = params.Exit_Time
        stop_loss_p = int(params.StopLoss)
        user_weekday = (params.Entry_Weekday).capitalize()
        max_entries = int(params.Max_Entries)
        Rolling_Straddle_Positional(option_data, expiry_key_list, entry_time, exit_time, stop_loss_p, user_weekday, max_entries)

In [9]:
%%time
filename, year = Filename(year_data, instrument)
root_path = data_root_path / instrument / "Option_Data" / filename
data_to_read = open(root_path, "rb")  # Opening Pickle file and Load into a Variable
option_data = pickle.load(data_to_read)
expiry_key_list = sorted(list(set(list(option_data.keys())) & set(dates_range)))

CPU times: total: 2.77 s
Wall time: 2.79 s


In [10]:
%%time
Driver_Code(option_data, expiry_key_list, param_data)

,User Entry Time,User Exit Time,user_weekday,Stop Loss %,Max Entries,Expiry,Entry Date_1,Exit Date_1,Entry Weekday_1,Exit Weekday_1,...,Entry Weekday_5,Exit Weekday_5,Entry Time_5,Exit Time_5,Exit Straddle_5,Stop Loss_5,SL Type_5,CE_PNL_5,PE_PNL_5,Total_PNL_5
0,09:20:59,15:20:59,Monday,20,5,2023-01-12,2023-01-09,2023-01-11,Monday,Wednesday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,09:20:59,15:20:59,Monday,20,5,2023-01-19,2023-01-16,2023-01-16,Monday,Monday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09:20:59,15:20:59,Monday,20,5,2023-01-25,2023-01-23,2023-01-25,Monday,Wednesday,...,Wednesday,Wednesday,11:26:59,15:29:59,47.549999,426.540015,Not Hit,135.050003,172.849991,307.899994
3,09:20:59,15:29:59,Monday,20,5,2023-02-09,2023-02-06,2023-02-09,Monday,Thursday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,09:20:59,15:20:59,Monday,20,5,2023-02-16,2023-02-13,2023-02-13,Monday,Monday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Donez !!!!!
CPU times: total: 172 ms
Wall time: 177 ms
